# PINN exploration to determine the acceleration due to gravity

In this notebook we define a simple example of a Physics-Informed Neural Network. We weill try to solve the following (very simple) problem:

Let us pretend that we are running an experiment where we throw a ball in the air and measure the height of the ball several times. We know that the height `u` of the ball at time `t` satisfies the following differential equation:


$$ \ddot{u} = -g $$

In other words, we assume that the acceleration due to gravity is constant, but we don't know the value of the constant `g`. We will use a PINN to model the solution to this PDE and determine the value of `g`. 


We will do this in the following steps:

1. Generate some synthetic data. Normally we would measure this with a sensor or device, but for this little tutorial we will just generate these measurements by using the exact solution and adding some noise (normally you won't know the exact solution of the problem!)

2. Define and train the neural network.





In [6]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Generate "Experimental" Data
torch.manual_seed(42)

t_data = torch.linspace(0, 2, 10).view(-1, 1)
true_g = 9.81
v0 = 10.0

noise_std = 0.2

# u(t) = v0*t - 0.5*g*t^2
u_data = v0 * t_data - 0.5 * true_g * t_data**2 + noise_std * torch.randn_like(t_data)


In [8]:
# Define the PINN. It's a simple NN that is used to model the value of the height. 

class GravityPINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32), nn.Tanh(),
            nn.Linear(32, 32), nn.Tanh(),
            nn.Linear(32, 1)
        )
        # We initialize g with a "wrong" guess (e.g., 5.0)
        self.g = nn.Parameter(torch.tensor([5.0], requires_grad=True))

    def forward(self, t):
        return self.net(t)

In [9]:
model = GravityPINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training points for the PDE (the "Bulk")
t_physics = torch.linspace(0, 2, 100).view(-1, 1).requires_grad_(True)

for epoch in range(20001):
    optimizer.zero_grad()
    
    # 1. Data Loss: Match the 10 noisy measurements
    u_pred = model(t_data)
    loss_data = torch.mean((u_pred - u_data)**2)
    
    # 2. Physics Loss: Enforce u'' = -g
    u_p = model(t_physics)
    
    # Get first derivative (velocity)
    u_t = torch.autograd.grad(u_p, t_physics, torch.ones_like(u_p), create_graph=True)[0]
    # Get second derivative (acceleration)
    u_tt = torch.autograd.grad(u_t, t_physics, torch.ones_like(u_t), create_graph=True)[0]
    
    # Our PDE residual: u'' + g = 0
    loss_physics = torch.mean((u_tt + model.g)**2)
    
    # 3. Total Loss
    total_loss = loss_data + loss_physics
    total_loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss {total_loss.item():.4f}, Inferred g: {model.g.item():.4f}")

print(f"\nFinal Inferred g: {model.g.item():.4f} (True g: 9.81)")

Epoch 0: Loss 37.1103, Inferred g: 4.9990
Epoch 500: Loss 1.2566, Inferred g: 4.7705
Epoch 1000: Loss 0.8726, Inferred g: 4.7972
Epoch 1500: Loss 0.8489, Inferred g: 4.8696
Epoch 2000: Loss 0.8198, Inferred g: 4.9694
Epoch 2500: Loss 0.7838, Inferred g: 5.0984
Epoch 3000: Loss 0.7398, Inferred g: 5.2590
Epoch 3500: Loss 0.6891, Inferred g: 5.4538
Epoch 4000: Loss 0.6329, Inferred g: 5.6818
Epoch 4500: Loss 0.5728, Inferred g: 5.9410
Epoch 5000: Loss 0.5110, Inferred g: 6.2271
Epoch 5500: Loss 0.4506, Inferred g: 6.5343
Epoch 6000: Loss 0.3935, Inferred g: 6.8557
Epoch 6500: Loss 0.3416, Inferred g: 7.1845
Epoch 7000: Loss 0.2966, Inferred g: 7.5144
Epoch 7500: Loss 0.2580, Inferred g: 7.8403
Epoch 8000: Loss 0.2270, Inferred g: 8.1571
Epoch 8500: Loss 0.2037, Inferred g: 8.4549
Epoch 9000: Loss 0.1872, Inferred g: 8.7271
Epoch 9500: Loss 0.1764, Inferred g: 8.9675
Epoch 10000: Loss 0.1701, Inferred g: 9.1698
Epoch 10500: Loss 0.1669, Inferred g: 9.3287
Epoch 11000: Loss 0.1656, Inferre